# Playground
Assuming that semio was installed over the whl. In order to do so run,
`poetry build` in `src/python/core`


In [ ]:
import semio

In [ ]:
from semio.model import Representation, Any
Representation(body=Any(),lod=300,name="Body")

In [ ]:
from semio.model import Point,Pose,Sobject,Layout, Any
from semio.gateway import LayoutDesignRequest, GatewayProxy

layoutDesignRequest = LayoutDesignRequest(layout=Layout(
    sobjects=[
            Sobject(url="elements/RectangleWithMiter.gh",pose=Pose(point_of_view=Point(x=-400,y=10,z=-5),view=Quaternion(w=1,x=0,y=0,z=0)),parameters={'Length':Any(value=b'330')}),
            Sobject(url="elements/RectangleWithMiter.gh",pose=Pose(point_of_view=Point(x=30,y=500,z=20),view=Quaternion(w=0,x=0.707,y=-0.707,z=0)),parameters={'Length':Any(value=b'220')})
        ]
))
request

proxy = GatewayProxy()
proxy.LayoutDesign(request=layoutDesignRequest)